

QUESTIONS:
What is our 30-day all cause readmission rate across hospitals our members are admitted to?
What percent of patients have a PCP visit within a week of discharge?
Do either vary by individual or group PCP's?

BONUS:
Which individual or PCP groups should we make an extra effort with to build a relationship? Which have the highest volume?
What percent of our patients do not have a PCP assigned or haven't seen their PCP recently?
Are there certain patients we should focus on (e.g. frequent fliers)?
From this, what could a version 1 of an actionable dashboard or intervention list look like to support the Transition Care Managers?

NOTES:

discharge date + 30 days: readmission true or false
count trues

In [59]:
import pandas as pd
import datetime as dt

In [18]:
members = pd.read_csv('../Data Files/NSS_GreatCare_Members.csv', parse_dates=[61,62])
admissions = pd.read_csv('../Data Files/NSS_Hospital_Inpatient_Admissions.csv', parse_dates=[8,9])
pcp_visits = pd.read_csv('../Data Files/NSS_PrimaryCare_Office_Visits.csv', parse_dates=[12])

In [37]:
admissions[admissions['Patient ID'] == 'DS1|9994']

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,2015-11-23,...,1,1,21.0,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,2015-11-27,...,1,1,21.0,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0


In [33]:
total_admissions = admissions.shape[0]

In [80]:
readmissions_30 = 0
i = 0
for patient in admissions['Patient ID'].unique():
    patient_admissions = admissions[admissions['Patient ID']==patient]
    times_admitted = patient_admissions.shape[0]
    if tim
    try:
        patient_admissions.sort_values(by = 'Encounter End Date', axis = 1, inplace=True)
    except:
        print(patient)
#     print(find_initial_events(patient_admissions))
#     if times_admitted > 1 and readmission(patient, patient_admissions.iloc[0,9]):
#         readmissions_30 += 1
        

DS1|10033
DS1|10043
DS1|10073
DS1|10101
DS1|10118
DS1|10126
DS1|10130
DS1|10132
DS1|10135
DS1|10168
DS1|10169
DS1|10191
DS1|10193
DS1|10228
DS1|10239
DS1|10244
DS1|10253
DS1|10291
DS1|10296
DS1|10303
DS1|10305
DS1|10325
DS1|10343
DS1|10349
DS1|10375
DS1|10399
DS1|10411
DS1|10415
DS1|10423
DS1|10436
DS1|10451
DS1|10452
DS1|10453
DS1|1045
DS1|10494
DS1|10503
DS1|10507
DS1|10509
DS1|10515
DS1|10520
DS1|10522
DS1|10529
DS1|10535
DS1|10544
DS1|10598
DS1|1060
DS1|10624
DS1|10666
DS1|10674
DS1|10698
DS1|1069
DS1|10707
DS1|10712
DS1|10713
DS1|1072
DS1|10734
DS1|10744
DS1|10750
DS1|10770
DS1|10786
DS1|10795
DS1|10798
DS1|10809
DS1|10815
DS1|10830
DS1|10839
DS1|10843
DS1|10857
DS1|10861
DS1|10862
DS1|10889
DS1|1088
DS1|10892
DS1|10894
DS1|10905
DS1|10917
DS1|10921
DS1|10926
DS1|10961
DS1|10963
DS1|10966
DS1|10976
DS1|10
DS1|11005
DS1|11014
DS1|11031
DS1|11034
DS1|11058
DS1|11062
DS1|11071
DS1|11076
DS1|11080
DS1|11082
DS1|11089
DS1|11092
DS1|11098
DS1|11106
DS1|11120
DS1|11134
DS1|11156
DS1|1118

DS1|21394
DS1|21395
DS1|21398
DS1|21437
DS1|21438
DS1|21476
DS1|21477
DS1|21478
DS1|21482
DS1|21498
DS1|214
DS1|21514
DS1|21520
DS1|21526
DS1|21551
DS1|21561
DS1|21571
DS1|21597
DS1|2160
DS1|21614
DS1|2162
DS1|21630
DS1|21631
DS1|21648
DS1|21649
DS1|21659
DS1|21687
DS1|21707
DS1|21718
DS1|2172
DS1|21730
DS1|21768
DS1|21769
DS1|21812
DS1|21818
DS1|21832
DS1|2184
DS1|21855
DS1|21863
DS1|21870
DS1|2187
DS1|21884
DS1|21894
DS1|21902
DS1|21921
DS1|21926
DS1|21928
DS1|21929
DS1|21951
DS1|21973
DS1|21981
DS1|2200
DS1|22022
DS1|22023
DS1|22031
DS1|22062
DS1|22069
DS1|22072
DS1|2208
DS1|22092
DS1|2210
DS1|22110
DS1|22129
DS1|22150
DS1|22181
DS1|22197
DS1|22198
DS1|22204
DS1|2220
DS1|22237
DS1|22239
DS1|22241
DS1|22264
DS1|22272
DS1|22274
DS1|22279
DS1|22283
DS1|22292
DS1|22296
DS1|22312
DS1|22316
DS1|22319
DS1|22322
DS1|22330
DS1|22337
DS1|22357
DS1|22366
DS1|2237
DS1|2238
DS1|22395
DS1|22408
DS1|22421
DS1|22429
DS1|22444
DS1|22449
DS1|2249
DS1|22512
DS1|22521
DS1|22524
DS1|22525
DS1|22528
DS1|

DS1|32347
DS1|32405
DS1|32440
DS1|32445
DS1|32446
DS1|32450
DS1|3246
DS1|32494
DS1|32495
DS1|32505
DS1|32507
DS1|3250
DS1|32512
DS1|3251
DS1|3254
DS1|32564
DS1|32579
DS1|32586
DS1|32598
DS1|32599
DS1|32628
DS1|32644
DS1|3264
DS1|32664
DS1|32683
DS1|3268
DS1|32691
DS1|32699
DS1|32725
DS1|32754
DS1|32766
DS1|32770
DS1|32775
DS1|32812
DS1|32814
DS1|32822
DS1|32842
DS1|32845
DS1|32854
DS1|32890
DS1|32909
DS1|32915
DS1|32925
DS1|32937
DS1|3294
DS1|32975
DS1|32994
DS1|3300
DS1|33013
DS1|33043
DS1|33052
DS1|33055
DS1|33059
DS1|33065
DS1|33078
DS1|33125
DS1|33135
DS1|3315
DS1|3316
DS1|33175
DS1|33195
DS1|33201
DS1|33203
DS1|33209
DS1|33217
DS1|33239
DS1|3323
DS1|33241
DS1|33262
DS1|33267
DS1|33280
DS1|33286
DS1|3328
DS1|332
DS1|33318
DS1|33322
DS1|33335
DS1|33343
DS1|33345
DS1|33353
DS1|33380
DS1|33387
DS1|33389
DS1|33404
DS1|33434
DS1|33465
DS1|33483
DS1|33484
DS1|33504
DS1|33514
DS1|33516
DS1|33530
DS1|33535
DS1|33548
DS1|3354
DS1|3356
DS1|33637
DS1|33639
DS1|33667
DS1|33673
DS1|33678
DS1|33

DS1|4383
DS1|4384
DS1|43854
DS1|43857
DS1|43922
DS1|43954
DS1|43978
DS1|43993
DS1|44005
DS1|44006
DS1|44018
DS1|44031
DS1|44120
DS1|44140
DS1|44147
DS1|44151
DS1|44152
DS1|44157
DS1|44163
DS1|4416
DS1|44172
DS1|44182
DS1|44188
DS1|44206
DS1|4422
DS1|44246
DS1|44268
DS1|44271
DS1|44285
DS1|44289
DS1|44293
DS1|44313
DS1|4432
DS1|44336
DS1|44343
DS1|4434
DS1|44370
DS1|44371
DS1|44375
DS1|44377
DS1|44381
DS1|44388
DS1|44389
DS1|44398
DS1|44401
DS1|44405
DS1|44443
DS1|44476
DS1|44488
DS1|44499
DS1|44503
DS1|44516
DS1|44536
DS1|44563
DS1|44569
DS1|44576
DS1|44577
DS1|44618
DS1|44635
DS1|44669
DS1|4468
DS1|44691
DS1|44699
DS1|44705
DS1|44744
DS1|44752
DS1|4475
DS1|44762
DS1|44812
DS1|44823
DS1|44838
DS1|44840
DS1|44843
DS1|4484
DS1|44886
DS1|44890
DS1|44914
DS1|44916
DS1|44924
DS1|44939
DS1|44946
DS1|44967
DS1|44974
DS1|44979
DS1|44981
DS1|449
DS1|4500
DS1|45025
DS1|45042
DS1|45051
DS1|45052
DS1|45073
DS1|45074
DS1|45079
DS1|45083
DS1|45089
DS1|45092
DS1|45100
DS1|45104
DS1|45106
DS1|45117
DS

DS1|55119
DS1|55122
DS1|55140
DS1|55145
DS1|55147
DS1|55171
DS1|55187
DS1|55188
DS1|5518
DS1|55201
DS1|55208
DS1|5521
DS1|55223
DS1|55235
DS1|55242
DS1|55259
DS1|55281
DS1|55289
DS1|55305
DS1|55342
DS1|55358
DS1|55360
DS1|55362
DS1|55375
DS1|55379
DS1|55395
DS1|55401
DS1|55402
DS1|55412
DS1|55447
DS1|5544
DS1|55472
DS1|55484
DS1|55486
DS1|55490
DS1|55504
DS1|55537
DS1|55555
DS1|55558
DS1|55587
DS1|55589
DS1|55595
DS1|55609
DS1|5562
DS1|55655
DS1|55680
DS1|55689
DS1|5568
DS1|55696
DS1|55708
DS1|55735
DS1|55739
DS1|55779
DS1|55788
DS1|55793
DS1|55867
DS1|55868
DS1|5586
DS1|55877
DS1|55928
DS1|55929
DS1|55965
DS1|55968
DS1|55974
DS1|55987
DS1|55
DS1|56008
DS1|56022
DS1|56034
DS1|56036
DS1|56047
DS1|56068
DS1|56076
DS1|56083
DS1|56093
DS1|560
DS1|5610
DS1|56130
DS1|56133
DS1|56137
DS1|56140
DS1|56176
DS1|5620
DS1|56220
DS1|56245
DS1|56261
DS1|56263
DS1|5630
DS1|56310
DS1|56312
DS1|56324
DS1|56329
DS1|56340
DS1|56349
DS1|56350
DS1|56352
DS1|56358
DS1|56375
DS1|56377
DS1|5637
DS1|56393
DS1|5

In [69]:
def readmission(patient, initial_discharge):
    thirty_days = initial_discharge + dt.timedelta(days=30)
    timeframe_mask = (admissions['Patient ID'] == patient) & (admissions['Encounter Start Date'] > initial_discharge) & (admissions['Encounter Start Date'] < thirty_days)
    if admissions[timeframe_mask].shape[0] > 0:
        return True

In [72]:
def find_initial_events(patient_admissions):
    #initial_events dict has admission date for keys and readmission status (bool) for values
    initial_events = {}
    initial_discharge = None
    for ind, row in patient_admissions.iterrows():
        if initial_discharge == None:
            initial_discharge = row['Encounter End Date']
            initial_events[row['Encounter Start Date']] = False
        elif row['Encounter Start Date'] <= initial_discharge + dt.timedelta(days=30):
            initial_events[row['Encounter Start Date']] = True
        else:
            initial_discharge = row['Encounter End Date']
            initial_events[row['Encounter Start Date']] = False
    return initial_events
            
    

In [71]:
admissions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6213 entries, 0 to 6212
Data columns (total 23 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   Encounter ID                                         6213 non-null   object        
 1   Patient ID                                           6213 non-null   object        
 2   Encounter Primary Payor ID                           6213 non-null   object        
 3   Encounter Primary Plan ID                            6213 non-null   object        
 4   Facility Provider ID                                 6213 non-null   object        
 5   Attending Provider ID                                5884 non-null   object        
 6   Attending Provider NPI                               5884 non-null   float64       
 7   Facility Provider NPI                                6213 non-null   object        
 8 